# Notebook 01: Collision Theory

## 1. Introduction to Reaction Dynamics

Chemical kinetics tells us *how fast* a reaction occurs (the rate constant, $k$). **Reaction dynamics** asks *why* it occurs at that rate and *how* the individual molecular events unfold. It bridges the gap between the macroscopic world of bulk concentrations and the microscopic world of molecular collisions and energy transfer.

In this notebook, we explore **Collision Theory**, the most fundamental model for understanding gas-phase reactions. The core idea is simple: for a reaction to occur, molecules must satisfy three conditions (the "Simplest Model"):

1.  **Collision**: They must hit each other (obviously!).
2.  **Energy**: They must collide with enough kinetic energy to break bonds ($E \ge E_a$).
3.  **Orientation**: They must collide with the correct geometry (steric requirement).

### Learning Objectives
By the end of this notebook, you should be able to:
1.  Calculate collision frequencies and collision densities for gas-phase molecules.
2.  Derive the expression for the rate constant from collision theory.
3.  Compare theoretical pre-exponential factors with experimental data to understand **steric factors**.
4.  Explain the "Harpoon Mechanism" for electron transfer reactions.
5.  Describe the RRK model for unimolecular decay.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
from scipy import constants
import ipywidgets as widgets

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

# Constants
k_B = constants.Boltzmann
N_A = constants.Avogadro
R = constants.gas_constant

print("Libraries loaded successfully!")

## 2. The Collision Rate

### 2.1 Collision Frequency ($z$)
Consider a single molecule A moving through a gas of stationary B molecules. How many collisions does it make per second?

If molecule A has a diameter $d_A$ and B has diameter $d_B$, they collide if their centers come within a distance $d = \frac{1}{2}(d_A + d_B)$. We call $\sigma = \pi d^2$ the **collision cross-section**.

In time $\Delta t$, the molecule sweeps out a volume $V_{sweep} = \sigma v \Delta t$. The number of B molecules in this volume is $\mathcal{N}_B V_{sweep}$, where $\mathcal{N}_B$ is the number density (molecules/m$^3$).

However, B molecules are not stationary. We must use the **mean relative speed**, $\bar{v}_{rel}$. From the Maxwell-Boltzmann distribution:
$$ \bar{v}_{rel} = \sqrt{\frac{8 k_B T}{\pi \mu}} $$
where $\mu = \frac{m_A m_B}{m_A + m_B}$ is the reduced mass.

The collision frequency for a *single* A molecule is:
$$ z = \sigma \bar{v}_{rel} \mathcal{N}_B $$

![Collision Cylinder](images/collision_cylinder.png)

### 2.2 Collision Density ($Z_{AB}$)
The total number of collisions per unit volume per unit time is the **collision density**, $Z_{AB}$. Since there are $\mathcal{N}_A$ molecules of A per unit volume:
$$ Z_{AB} = z \mathcal{N}_A = \sigma \bar{v}_{rel} \mathcal{N}_A \mathcal{N}_B $$

For like molecules (A + A collisions), we must divide by 2 to avoid double counting:
$$ Z_{AA} = \frac{1}{2} \sigma \bar{v}_{rel} \mathcal{N}_A^2 $$

### Interactive Calculator
Use the tool below to see how temperature, pressure, and size affect the collision rate.

In [ ]:
class CollisionCalculator:
    """Interactive calculator for collision theory parameters"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
    
    def setup_widgets(self):
        """Create interactive widgets"""
        
        # Temperature slider
        self.temp_slider = widgets.FloatSlider(
            value=298,
            min=200,
            max=1000,
            step=10,
            description='T (K):',
            continuous_update=False
        )
        
        # Pressure slider
        self.pressure_slider = widgets.FloatSlider(
            value=1.0,
            min=0.1,
            max=10.0,
            step=0.1,
            description='p (bar):',
            continuous_update=False
        )
        
        # Cross-section slider
        self.sigma_slider = widgets.FloatSlider(
            value=0.43,
            min=0.1,
            max=2.0,
            step=0.05,
            description='σ (nm²):',
            continuous_update=False
        )
        
        # Molar mass slider
        self.mass_slider = widgets.FloatSlider(
            value=28.0,
            min=2.0,
            max=200.0,
            step=1.0,
            description='M (g/mol):',
            continuous_update=False
        )
        
        # Calculate button
        self.calc_button = widgets.Button(
            description='Calculate!',
            button_style='success',
            icon='calculator'
        )
        
        self.calc_button.on_click(self.calculate)
        
    def calculate(self, b):
        """Calculate and display collision parameters"""
        
        T = self.temp_slider.value
        p = self.pressure_slider.value * 1e5  # Convert to Pa
        sigma = self.sigma_slider.value * 1e-18  # Convert to m²
        M = self.mass_slider.value * 1e-3  # Convert to kg/mol
        
        # Calculate concentration
        c = p / (R * T)  # mol/m³
        
        # Calculate mean speed
        v_mean = np.sqrt(8 * R * T / (np.pi * M))
        
        # Calculate collision frequency for single molecule
        z = sigma * v_mean * c * N_A
        
        # Calculate collision density
        Z_AA = 0.5 * sigma * v_mean * (c * N_A)**2
        
        # Display results
        print("\n" + "="*60)
        print("COLLISION THEORY RESULTS")
        print("="*60)
        print(f"Temperature:           {T:.1f} K")
        print(f"Pressure:              {p/1e5:.2f} bar")
        print(f"Concentration:         {c:.2f} mol/m³")
        print(f"Mean speed:            {v_mean:.1f} m/s")
        print(f"Cross-section:         {sigma*1e18:.2f} nm²")
        print(f"\n🎯 COLLISION FREQUENCY (single molecule):")
        print(f"   z = {z:.2e} collisions/s")
        print(f"\n🎯 COLLISION DENSITY:")
        print(f"   Z_AA = {Z_AA:.2e} collisions/(m³·s)")
        print("="*60)
        
        # Create visualization
        self.plot_temperature_dependence(sigma, M)
    
    def plot_temperature_dependence(self, sigma, M):
        """Plot how collision parameters vary with temperature"""
        
        T_range = np.linspace(200, 1000, 100)
        v_mean = np.sqrt(8 * R * T_range / (np.pi * M))
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # Mean speed vs temperature
        ax1.plot(T_range, v_mean, 'b-', linewidth=2)
        ax1.axvline(self.temp_slider.value, color='r', 
                    linestyle='--', label='Current T')
        ax1.set_xlabel('Temperature (K)', fontsize=12)
        ax1.set_ylabel('Mean Speed (m/s)', fontsize=12)
        ax1.set_title('Temperature Dependence of Mean Speed')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Collision frequency vs temperature
        p = self.pressure_slider.value * 1e5
        c = p / (R * T_range)
        z = sigma * v_mean * c * N_A
        
        ax2.semilogy(T_range, z, 'g-', linewidth=2)
        ax2.axvline(self.temp_slider.value, color='r', 
                    linestyle='--', label='Current T')
        ax2.set_xlabel('Temperature (K)', fontsize=12)
        ax2.set_ylabel('Collision Frequency (s⁻¹)', fontsize=12)
        ax2.set_title('Temperature Dependence of Collision Frequency')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def setup_display(self):
        """Display the interactive interface"""
        
        display(widgets.VBox([
            widgets.HTML("<h3>🔬 Interactive Collision Calculator</h3>"),
            self.temp_slider,
            self.pressure_slider,
            self.sigma_slider,
            self.mass_slider,
            self.calc_button
        ]))

# Create and display calculator
calculator = CollisionCalculator()

### ❓ Concept Check

<details>
<summary><strong>Q: Why does the collision frequency $Z_{AA}$ depend on the square of the density ($N^2$)?</strong> (Click to reveal)</summary>

Because a collision requires *two* molecules. If you double the concentration, you double the number of potential targets *and* double the number of projectiles, leading to a four-fold increase in collisions.
</details>

## 3. Energy Requirements and the Rate Constant

If every collision led to a reaction, rates would be astronomically high ($~10^{10}$ M$^{-1}$s$^{-1}$). In reality, most collisions are elastic (they bounce off). Reaction requires energy to break bonds.

### 3.1 The Energy Barrier
We assume a collision is successful only if the kinetic energy along the line of centers exceeds a threshold $\varepsilon_a$. Integrating the Boltzmann distribution over energies greater than $\varepsilon_a$ yields the theoretical rate constant:
$$ k_{theory} = \sigma \bar{v}_{rel} N_A e^{-E_a/RT} $$

Comparing this to the Arrhenius equation ($k = A e^{-E_a/RT}$), we identify the theoretical pre-exponential factor:
$$ A_{theory} = \sigma \bar{v}_{rel} N_A $$

### 3.2 Energy-Dependent Cross-Section $\sigma(\varepsilon)$
A more sophisticated view treats the cross-section as a function of energy. If the energy is too low, the molecules bounce ($\sigma=0$). If high enough, they might react.

For a simple "hard sphere" model with a threshold:
$$ \sigma(\varepsilon) = \begin{cases} 0 & \varepsilon < \varepsilon_a \\ \sigma \left(1 - \frac{\varepsilon_a}{\varepsilon}\right) & \varepsilon \ge \varepsilon_a \end{cases} $$

This explains why the cross-section "turns on" at the threshold and approaches the geometric limit $\sigma$ at very high energies.

In [ ]:
def interactive_cross_section():
    """Interactive visualization of σ(ε)"""
    
    @widgets.interact(
        E_a=widgets.FloatSlider(min=0, max=200, step=10, value=50,
                                description='Eₐ (kJ/mol)'),
        sigma_0=widgets.FloatSlider(min=0.1, max=2, step=0.1, value=0.5,
                                     description='σ₀ (nm²)')
    )
    def plot_sigma_epsilon(E_a, sigma_0):
        
        # Convert to J
        E_a_J = E_a * 1000 / N_A
        
        # Energy range
        epsilon = np.linspace(0, 5*E_a_J, 500)
        
        # Calculate σ(ε)
        sigma = np.zeros_like(epsilon)
        mask = epsilon >= E_a_J
        sigma[mask] = sigma_0 * (1 - E_a_J/epsilon[mask])
        
        # Plot
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(epsilon*N_A/1000, sigma, 'b-', linewidth=2, 
                label=r'$\sigma(\varepsilon) = \sigma_0(1 - \varepsilon_a/\varepsilon)$')
        ax.axvline(E_a, color='r', linestyle='--', 
                   label=f'Threshold: Eₐ = {E_a} kJ/mol')
        ax.axhline(sigma_0, color='g', linestyle=':', 
                   label=f'Maximum: σ₀ = {sigma_0} nm²')
        
        # Shade reactive region
        mask_plot = epsilon*N_A/1000 >= E_a
        ax.fill_between(epsilon*N_A/1000, 0, sigma, 
                        where=mask_plot, alpha=0.3, color='green',
                        label='Reactive region')
        
        ax.set_xlabel('Kinetic Energy (kJ/mol)', fontsize=12)
        ax.set_ylabel('Cross-section (nm²)', fontsize=12)
        ax.set_title('Energy-Dependent Collision Cross-Section', fontsize=14)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, sigma_0*1.2)
        
        plt.tight_layout()
        plt.show()

print("\n📊 Interactive Cross-Section Visualization:")
interactive_cross_section()

## 4. Visualizing Collisions

To better understand the steric factor and collision dynamics, let's visualize a 2D collision. The "Impact Parameter" ($b$) is the perpendicular distance between the path of the projectile and the center of the target.
-   **Head-on collision**: $b=0$
-   **Glancing collision**: $b \approx d_A + d_B$
-   **Miss**: $b > d_A + d_B$

In [ ]:
class CollisionAnimator:
    """Animate molecular collisions"""
    
    def __init__(self):
        self.fig, self.ax = plt.subplots(figsize=(8, 8))
        self.setup_plot()
    
    def setup_plot(self):
        """Setup the animation plot"""
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        self.ax.set_aspect('equal')
        self.ax.set_xlabel('x (arbitrary units)')
        self.ax.set_ylabel('y (arbitrary units)')
        self.ax.set_title('Molecular Collision Simulation')
        self.ax.grid(True, alpha=0.3)
    
    def animate_collision(self, impact_parameter=0):
        """Animate a collision with given impact parameter"""
        
        # Particle A (stationary at origin)
        R_A = 1.0
        circle_A = plt.Circle((0, 0), R_A, color='blue', alpha=0.5, label='Target')
        self.ax.add_patch(circle_A)
        
        # Particle B (moving)
        R_B = 1.0
        v_B = 2.0  # velocity
        
        # Initial position
        x_init = -5
        y_init = impact_parameter
        
        # Trajectory
        times = np.linspace(0, 5, 100)
        x_traj = x_init + v_B * times
        y_traj = np.full_like(times, y_init)
        
        # Check for collision
        distances = np.sqrt(x_traj**2 + y_traj**2)
        collision_occurs = np.min(distances) <= (R_A + R_B)
        
        # Plotting
        line, = self.ax.plot([], [], 'r--', alpha=0.5, label='Trajectory')
        circle_B = plt.Circle((x_init, y_init), R_B, color='red', alpha=0.5, label='Projectile')
        self.ax.add_patch(circle_B)
        
        def init():
            line.set_data([], [])
            return line, circle_B
        
        def update(frame):
            line.set_data(x_traj[:frame], y_traj[:frame])
            circle_B.center = (x_traj[frame], y_traj[frame])
            
            if collision_occurs and frame > 50:
                circle_B.set_color('green')
                self.ax.set_title('Collision Occurred! ✓')
            else:
                circle_B.set_color('red')
            
            return line, circle_B
        
        anim = FuncAnimation(self.fig, update, frames=len(times),
                           init_func=init, interval=50, blit=True)
        
        plt.close() # Prevent static plot from showing
        return HTML(anim.to_jshtml())

# Create interactive collision animator
print("\n🎬 Collision Animation:")

@widgets.interact(
    impact_parameter=widgets.FloatSlider(
        min=0, max=3, step=0.1, value=1.0,
        description='Impact b:'
    )
)
def show_collision(impact_parameter):
    animator = CollisionAnimator()
    return animator.animate_collision(impact_parameter)

## 5. The Harpoon Mechanism

Some reactions, like K + Br$_2$ $\rightarrow$ KBr + Br, occur *faster* than the collision rate calculated from their physical size. This implies a "giant" cross-section.

### Mechanism
1.  **Electron Jump**: The alkali metal (K) has a low ionization energy ($I$). The halogen (Br$_2$) has a high electron affinity ($E_{ea}$). At a certain distance $R^*$, an electron tunnels from K to Br$_2$.
2.  **Coulombic Attraction**: The resulting ions K$^+$ and Br$_2^-$ attract each other strongly via the Coulomb force.
3.  **Harpoon**: This electrostatic force acts like a "harpoon," pulling the molecules together even if they would have missed each other physically.

![Harpoon Mechanism](images/harpoon_mechanism.png)

### Critical Distance $R^*$
The electron transfer becomes energetically favorable when the Coulombic stabilization overcomes the energy cost of ionization:
$$ \Delta E = I - E_{ea} - \frac{e^2}{4\pi\varepsilon_0 R^*} = 0 $$
Solving for $R^*$:
$$ R^* = \frac{e^2}{4\pi\varepsilon_0 (I - E_{ea})} $$

In [ ]:
def harpoon_mechanism():
    """Interactive exploration of harpoon mechanism"""
    
    @widgets.interact(
        I=widgets.FloatSlider(min=3, max=6, step=0.1, value=4.2,
                             description='I (eV):',
                             style={'description_width': 'initial'}),
        E_ea=widgets.FloatSlider(min=1, max=3, step=0.1, value=1.2,
                                description='Eₑₐ (eV):',
                                style={'description_width': 'initial'}),
        d_sum=widgets.FloatSlider(min=200, max=600, step=50, value=400,
                                 description='Σradii (pm):',
                                 style={'description_width': 'initial'})
    )
    def plot_harpoon(I, E_ea, d_sum):
        
        # Convert eV to J
        I_J = I * constants.elementary_charge
        E_ea_J = E_ea * constants.elementary_charge
        
        # Distance range (m)
        R = np.linspace(0.2e-9, 3e-9, 500)
        
        # Energy changes
        E_ion = I_J - E_ea_J  # Ionization - electron affinity
        E_coulomb = -constants.elementary_charge**2 / (4*np.pi*constants.epsilon_0*R)
        E_total = E_ion + E_coulomb
        
        # Find critical distance R*
        idx_cross = np.where(E_total <= 0)[0]
        if len(idx_cross) > 0:
            R_star = R[idx_cross[0]]
            P_factor = (R_star / (d_sum*1e-12))**2
        else:
            R_star = None
            P_factor = None
        
        # Plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Energy diagram
        ax1.axhline(0, color='k', linestyle='-', linewidth=0.5)
        ax1.plot(R*1e9, E_ion*np.ones_like(R)*constants.e, 'b--', 
                label=f'Ionization cost: {I-E_ea:.2f} eV')
        ax1.plot(R*1e9, E_coulomb/constants.elementary_charge, 'r--', 
                label='Coulombic attraction')
        ax1.plot(R*1e9, E_total/constants.elementary_charge, 'g-', linewidth=2,
                label='Total energy')
        
        if R_star:
            ax1.axvline(R_star*1e9, color='purple', linestyle=':', linewidth=2,
                       label=f'R* = {R_star*1e9:.2f} nm')
            ax1.axhline(0, xmin=0, xmax=R_star*3e9/ax1.get_xlim()[1],
                       color='green', alpha=0.3, linewidth=5)
        
        ax1.set_xlabel('Distance R (nm)', fontsize=12)
        ax1.set_ylabel('Energy (eV)', fontsize=12)
        ax1.set_title('Harpoon Mechanism Energy Profile', fontsize=14)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(-5, 5)
        
        # Steric factor diagram
        if P_factor:
            angles = np.linspace(0, 2*np.pi, 100)
            
            # Simple collision
            x_simple = d_sum * np.cos(angles)
            y_simple = d_sum * np.sin(angles)
            ax2.plot(x_simple, y_simple, 'b-', linewidth=2, label='Simple collision')
            ax2.fill(x_simple, y_simple, color='blue', alpha=0.2)
            
            # Harpoon collision
            x_harpoon = R_star*1e12 * np.cos(angles)
            y_harpoon = R_star*1e12 * np.sin(angles)
            ax2.plot(x_harpoon, y_harpoon, 'r-', linewidth=2, label='Harpoon mechanism')
            ax2.fill(x_harpoon, y_harpoon, color='red', alpha=0.2)
            
            ax2.set_xlabel('x (pm)', fontsize=12)
            ax2.set_ylabel('y (pm)', fontsize=12)
            ax2.set_title(f'Cross-Section Comparison\nP = {P_factor:.2f}', fontsize=14)
            ax2.legend()
            ax2.set_aspect('equal')
            ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print results
        if R_star:
            print(f"\n{'='*60}")
            print(f"HARPOON MECHANISM ANALYSIS")
            print(f"{'='*60}")
            print(f"Ionization energy:        {I:.2f} eV")
            print(f"Electron affinity:        {E_ea:.2f} eV")
            print(f"Net cost:                 {I-E_ea:.2f} eV")
            print(f"\n🎯 Critical distance:      R* = {R_star*1e9:.3f} nm = {R_star*1e12:.1f} pm")
            print(f"🎯 Simple collision σ:     π({d_sum/2:.0f} pm)² = {np.pi*(d_sum/2)**2:.0f} pm²")
            print(f"🎯 Harpoon collision σ:    π({R_star*1e12:.0f} pm)² = {np.pi*(R_star*1e12)**2:.0f} pm²")
            print(f"\n✨ STERIC FACTOR:          P = {P_factor:.2f}")
            print(f"{'='*60}")
        else:
            print("\n⚠️  No harpoon mechanism: electron transfer not favorable")

print("\n⚡ Harpoon Mechanism Explorer:")
harpoon_mechanism()

## 6. The RRK Model (Unimolecular Reactions)

How does a single molecule A$^*$ break apart? It has excess energy, but that energy is distributed among many vibrational modes. For reaction to occur, enough energy ($E^*$) must concentrate in the specific bond that breaks.

The Rice-Ramsperger-Kassel (RRK) model treats the molecule as a collection of $s$ coupled harmonic oscillators. The probability of finding energy $\ge E^*$ in one specific mode, given total energy $E$, is:
$$ P(E \ge E^*) = \left(1 - \frac{E^*}{E}\right)^{s-1} $$

The rate constant $k(E)$ is proportional to this probability:
$$ k(E) = k_b \left(1 - \frac{E^*}{E}\right)^{s-1} $$
where $k_b$ is the rate of energy redistribution (approx vibrational frequency).

**Key Insight**: Larger molecules (larger $s$) react slower for the same energy density because the energy has more places to "hide" (entropy effect).

In [ ]:
def rrk_model_explorer():
    """Interactive RRK model visualization"""
    
    @widgets.interact(
        s=widgets.IntSlider(min=3, max=30, step=1, value=10,
                           description='s (oscillators):'),
        E_ratio=widgets.FloatSlider(min=1.0, max=5.0, step=0.1, value=1.25,
                                    description='E/E*:')
    )
    def plot_rrk(s, E_ratio):
        
        # Calculate P
        if E_ratio > 1:
            P = (1 - 1/E_ratio)**(s-1)
        else:
            P = 0
        
        # Plot vs E/E* for different s values
        E_ratios = np.linspace(1.0, 5.0, 200)
        s_values = [5, 10, 20]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # P vs E/E* for different s
        for s_val in s_values:
            P_vals = np.maximum((1 - 1/E_ratios)**(s_val-1), 0)
            ax1.plot(E_ratios, P_vals, linewidth=2, label=f's = {s_val}')
        
        # Current point
        ax1.plot(E_ratio, P, 'ro', markersize=10, label='Current')
        
        ax1.set_xlabel('E/E*', fontsize=12)
        ax1.set_ylabel('P = (1 - E*/E)^(s-1)', fontsize=12)
        ax1.set_title('RRK Probability vs Energy Ratio', fontsize=14)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, 1.1)
        
        # P vs s for current E/E*
        s_range = np.arange(3, 31)
        P_vs_s = (1 - 1/E_ratio)**(s_range-1)
        
        ax2.plot(s_range, P_vs_s, 'b-', linewidth=2)
        ax2.plot(s, P, 'ro', markersize=10, label='Current')
        ax2.set_xlabel('Number of oscillators (s)', fontsize=12)
        ax2.set_ylabel('P = (1 - E*/E)^(s-1)', fontsize=12)
        ax2.set_title(f'Effect of Molecule Size (E/E* = {E_ratio:.2f})', fontsize=14)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Display result
        print(f"\n{'='*60}")
        print(f"RRK MODEL CALCULATION")
        print(f"{'='*60}")
        print(f"Number of oscillators:  s = {s}")
        print(f"Energy ratio:           E/E* = {E_ratio:.2f}")
        print(f"\n🎯 Reaction probability: P = {P:.4f}")
        print(f"\n💡 Interpretation:")
        if P < 0.01:
            print("   Very unlikely to react - energy too dispersed")
        elif P < 0.1:
            print("   Low probability - some energy localization needed")
        elif P < 0.5:
            print("   Moderate probability - reasonable energy localization")
        else:
            print("   High probability - good energy localization")
        print(f"{'='*60}")

print("\n🎲 RRK Model Explorer:")
rrk_model_explorer()

## Summary

1.  **Collision Theory**: Rate = Collision Frequency $\times$ Steric Factor $\times$ Energy Factor.
2.  **Cross-Section**: Effective target area, depends on kinetic energy.
3.  **Harpoon Mechanism**: Electron transfer increases the effective cross-section via Coulombic attraction.
4.  **RRK Model**: Unimolecular rates depend on how energy is statistically distributed among vibrational modes.

## Practice Problem
**Calculate the pre-exponential factor for H$_2$ + I$_2$ $\rightarrow$ 2HI at 600 K.**

Given:
- Collision cross-section $\sigma \approx 0.30 \text{ nm}^2$
- Mass H$_2$ = 2.016 amu, Mass I$_2$ = 253.8 amu

**Steps:**
1.  Calculate reduced mass $\mu$.
2.  Calculate mean relative speed $\bar{v}_{rel}$.
3.  Calculate $A = \sigma \bar{v}_{rel} N_A$.

*(Check your answer: You should get approx $10^{11} \text{ dm}^3 \text{ mol}^{-1} \text{ s}^{-1}$)*